In [ ]:
import lateral_signaling as lsig

import os
from glob import glob

import numpy as np
import pandas as pd

import skimage
import skimage.io as io

import colorcet as cc
import bebi103

import bokeh.io
bokeh.io.output_notebook()

In [ ]:
# Reading
data_dir = os.path.abspath("../data/imaging/FACS_brightfield/")

# Writing
save = False
save_dir = os.path.abspath("../data/analysis/FACS_brightfield/")

# Set random seed
seed = 2022

In [ ]:
# Get TIFF files
imfiles = glob(os.path.join(data_dir, "*.tif*"))
imfiles = [os.path.realpath(f) for f in imfiles]

# Read images
load_im = lambda f: io.imread(f)
ims = io.ImageCollection(imfiles, load_func=load_im)

# Get density conditions
imnames = [os.path.split(f)[1].split(".")[0] for f in ims.files]

# Get image dimensions
ims = ims.concatenate()
n_ims, *imshape = ims.shape

In [ ]:
## Split images into sub-images, or windows
# Set edge length of square window
imsize = 240

# Fix random number generation using supplied seed
rng = np.random.default_rng(seed)

# Get number of windows that will fit inside each image
n_rows = imshape[0] // imsize
n_cols = imshape[1] // imsize

# Assemble all windows from all images into a list and keep track of the source image
im_windows = []
for im in ims:
    
    # Get windows and normalize intensity
    _windows = np.array([np.split(_hs, n_rows, axis=0) for _hs in np.split(im, n_cols, axis=1)])
    _windows = np.array([lsig.rescale_img(w) for w in _windows])
    
    # Randomize window order
    _shape = _windows.shape
    _windows = _windows.reshape(-1, *_shape[2:])
    _win_idx = np.arange(_shape[0] * _shape[1])
    rng.shuffle(_win_idx)
    _windows = _windows[_win_idx]

    # Store windows 
    im_windows.append(_windows)

# Sew together windows, alternating between images
windows = np.array(im_windows).transpose(1, 0, 2, 3).reshape(-1, *_shape[2:])
win_names = np.tile(imnames, n_rows * n_cols)

---

In [ ]:
# Initialize list to store cell boundary data
cbound_dfs = []

# Draw cell boundaries

To analyze each window, I change the value of `idx` to select a new random window, draw cell boundaries for 2-5 cells in the window, convert these vertices to a DataFrame, then append this to the list of DataFrames. 

Occasionally, an ROI was erroneously drawn. Because this is difficult to undo without deleting other ROIs drawn on the same image, I edit the misshapen ROI to have a small area - these ROIs are then filtered out later in the pipeline using an area cutoff.

In [ ]:
idx = 31

# Select image
im     = windows[idx]
imname = win_names[idx]
print(imname)

# Draw an ROI by clicking along the border of the well
cbounds_roicds = bebi103.image.draw_rois(im, flip=False, frame_height=800, cmap=cc.palette.gray)

In [ ]:
# Convert to DataFrame
df = bebi103.image.roicds_to_df(cbounds_roicds)
df["window"]  = idx
df["density"] = imname

cbound_dfs.append(df)

---

# Combine ROI data and save

In [ ]:
cbound_df = pd.concat(cbound_dfs)

if save:
    cbound_df.to_csv(os.path.join(save_dir, "cell_boundary_vertices.csv"))